# NLP Assignment #04 - Transformers to the rescue!
Deep Learning / Spring 1400, Khatam University

---

**Please pay attention to these notes:**
<br><br>


- **Assignment Due:** <b><font color='red'>1400.03.18</font></b> 23:59:00
- If you need any additional information, please review the assignment page on the course website.
- The items you need to answer are highlighted in <font color="SeaGreen">**bold SeaGreen**</font> and the coding parts you need to implement are denoted by:
```
#######################
# Your implementation #
#######################
```
- We always recommend co-operation and discussion in groups for assignments. However, **each student has to finish all the questions by him/herself**. If our matching system identifies any sort of copying, you'll be responsible for consequences.
- Students who audit this course should submit their assignments like other students to be qualified for attending the rest of the sessions.
- If you have any questions about this assignment, feel free to drop us a line. You may also post your questions on the course Microsoft Teams channel.
- You must run this notebook on Google Colab platform, it depends on Google Colab VM for some of the depencecies.
- You can double click on collapsed code cells to expand them.
- <b><font color='red'>When you are ready to submit, please follow the instructions at the end of this notebook.</font></b>


<br>



# Introduction
Enough RNNs! Transformers are here!

In this assignment, we are going to get a taste of what NLP became after the introduction of Transformers!

We will first simply fine-tune a pre-trained transformer-based language model (BERT) on MNLI, so you can compare its performance with your previous assignments. Then we will see even transformers are prone to some kind of errors. Also, we will get familiar with two different methods of interpreting the mysterious nature of transformers. 

<b><font color='red'>Keep in mind, in this notebook, you have freedome in your implementations.</font></b>

Let's begin!


# Fine-Tuning / Robustness

First things first! As an NLP researcher, you should know how to employ a Transformer-based model in a downstream task! A decent choice when it comes to Transformers is the [HuggingFace transformers](https://huggingface.co/transformers/index.html) library. You're free to choose your own tools of course, but we strongly recommend using the mentioned library as it provides a whole new world of utilities.

<b><font color="seaGreen">Fine-tune a BERT model on MNLI dataset!</font></b>

In [ ]:
# prepare dataset

#######################
# Your implementation #
#######################

In [ ]:
# create model architecture

#######################
# Your implementation #
#######################

In [ ]:
# train your model

#######################
# Your implementation #
#######################

In [ ]:
# evaluate on test set - create a confusion matrix

#######################
# Your implementation #
#######################

<b><font color="seaGreen">Compare your results with the baselines you had in previous assignments! Which class is the hardest for BERT to learn? Why?</font></b>

Alright, As you saw, Transformers show a remarkable performance compared to other architectures. Now you might think they have no weaknesses; As we will see, you could not be more wrong! A whole branch of research is devoted to generating adversarial examples to fool models, but here we are interested in "TextFooler" which is a simple greedy approach based on replacing the most important tokens with similar ones. Take a look at the [original paper](https://arxiv.org/pdf/1907.11932.pdf). We can find an implementation of `textfooler` in [this repository](https://github.com/QData/TextAttack). Let's install this tool:

In [ ]:
from IPython.display import clear_output

!pip install textattack

clear_output()
print("Done! Please restart the runtime.")

Now let's see some examples of fine-tuned BERT on MNLI getting fooled by simply replacing few words by running the following cell (it may take few minutes)!

In [ ]:
!textattack attack --model bert-base-uncased-mnli --recipe textfooler --num-examples 50

As you can see, BERT could be easily fooled! One thing to note is that there are both intuitive and counter-intuitive changes among the generated adversarial examples. <b><font color="seaGreen">Find a sample of each and for the intuitive example, explain why you think this way?</font></b>

<b><font color="red">Counter-intuitive change:</font></b></br>
<font color="red">Premise: <--Write Here--> </font></br>
<font color="red">Hypothesis: <--Write Here--> </font></br>
<b><font color="red">Changed to: </font></b></br>
<font color="red">Premise: <--Write Here--> </font></br>
<font color="red">Hypothesis: <--Write Here--> </font></br>

<b><font color="red">Intuitive change:</font></b></br>
<font color="red">Premise: <--Write Here--> </font></br>
<font color="red">Hypothesis: <--Write Here--> </font></br>
<b><font color="red">Changed to: </font></b></br>
<font color="red">Premise: <--Write Here--> </font></br>
<font color="red">Hypothesis: <--Write Here--> </font></br>

<b><font color="seaGreen">Why do you think this change is intuitive?</font></b>


<font color="red"> <--Write Your Answer Here--> </font>


<b><font color="seaGreen">What do you propose to make BERT more robust to these kind of attacks?</font></b>


<font color="red"> <--Write Your Answer Here--> </font>


# Masked Language Model Interpretation

As the black box models get more complex, *interpratation* and *interpretability* become more important. *Transformers* are a clear example of such models which had many researchs towards explaining their performance. Herein, the method we are going to (implement and) use is **Erasure**. In the [paper](https://arxiv.org/pdf/1612.08220.pdf) that presented this method, it is described as:

> *a general methodology to analyze and interpret decisions from a
neural model by observing the effects on the model of erasing various parts of the representation, such as input word-vector dimensions, intermediate hidden units, or input words.* 

![MLM](https://www.researchgate.net/publication/340223686/figure/fig1/AS:873545626308609@1585280915468/BERT-Original-sentence-how-are-you-doing-today_W640.jpg)

As you know, the well-known transformers that pushed NLP forward (BERT, RoBERTa, GPT2), are lanuage models by nature. So we apply *erasure* to a **Masked Language Model (MLM)** to interpret the model while doing its original task. We limit erasure to input tokens for simplicity. To find out which parts of the input (tokens) have the most impact on the final decision, we need to take these simple steps:

1. Prepare a text input with a single mask token. e.g. `My little [MASK] is taller than me.`
2. Let our MLM predict the most probable words to fill the mask: `brother, sister, friend, ...`
3. Choose one of the most probable words as target word. e.g. `sister`
4. Generate perturbed inputs, by erasing context tokens one by one. e.g. `My little [MASK] is [PAD] than me.` You can remove the word or replace it with pad token or ...
5. Feed each perturbed input to MLM and measure the change in probability of the target word.
6. Finally, the probability change caused by removing each token is considered as the **importance** of that token. 

The implementation of above steps is up to you. Obviously, you have no better choice than the widely used `transformers` library, but there are also choices within that:

You can use either `*ForMaskedLM` or `FillMaskPipeline` to implement erasure as explained. The former is a transformer `Model` with a language modeling head on top. You can pre-train/fine-tune a `Model` or use it for inference. On the other hand, `Pipeline`s are a great and easy way to use models only for inference. As the name suggests, pipelines run the whole process (including preprocessing) from the input text to the final output, offering ease of use at the 
cost of inflexibility.

<b><font color="seaGreen">Implement *erasure* as explained. You are free to break it to multiple functions, but the `compute_importances` function should pack the whole process and return importance scores as described in the docstring. Your implementation should work with any MLM in `transformers`.</font></b>

In [ ]:
# load tokenizer/model/pipeline

#######################
# Your implementation #
#######################

In [ ]:
# fill mask with top-10 (tokens, probs)

#######################
# Your implementation #
#######################

In [ ]:
# remove tokens one by one, returns a batch of inputs

#######################
# Your implementation #
#######################

In [ ]:
# compute token importances towards predicting a target word
def compute_importances(text, target_word_index=0):
"""
Computes importance scores for tokens in the input text. The target model is assumed to
be gloabl. If you want to change the function signature and pass the model as argument,
remember to fix the function calls in the visulization cells bellow.

Args:
  text: raw text with exactly one mask token
  target_word_index: the rank of target word in the top predictions (i-th most probable word)

Returns:
  tokens: tokenized input text in which the mask token is replaced with target word. (list of str)
  importances: list of importance values for each token. Its value for the target token is always zero.

"""
  #######################
  # Your implementation #
  #######################

  return tokens, importances

In [ ]:
# @title visualize importance
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as clr
from pylab import rcParams

from IPython.display import HTML

def visualize_attention(sentences, score_lists, color_maps='RdYlGn', rtl=False,
                        alpha=0.5, font_size=14, token_sep=' ', sentence_sep='<br/><br/>'):

  if type(color_maps) is str:
    color_maps = [color_maps] * len(sentences)

  span_sentences, style_sentences = [], []

  for s, tokens in enumerate(sentences):

    scores = score_lists[s]
    cmap = cm.get_cmap(color_maps[s])
    
    max_value = max(abs(min(scores)), abs(max(scores)))
    normer = clr.Normalize(vmin=-max_value/alpha, vmax=max_value/alpha)
    colors = [clr.to_hex(cmap(normer(x))) for x in scores]

    if len(tokens) != len(colors):
        raise ValueError("number of tokens and colors don't match")

    style_elems, span_elems = [], []
    for i in range(len(tokens)):
        style_elems.append(f'.c{s}-{i} {{ background-color: {colors[i]}; }}')
        span_elems.append(f'<span class="c{s}-{i}">{tokens[i]} </span>')

    span_sentences.append(token_sep.join(span_elems))
    style_sentences.append(' '.join(style_elems))
    text_dir = 'rtl' if rtl else 'ltr'

  return HTML(f"""<html><head><link href="https://fonts.googleapis.com/css?family=Roboto+Mono&display=swap" rel="stylesheet">
               <style>span {{ font-family: "Roboto Mono", monospace; font-size: {font_size}px; padding: 2px}} {' '.join(style_sentences)}</style>
               </head><body><p dir="{text_dir}">{sentence_sep.join(span_sentences)}</p></body></html>""")

In [ ]:
#@title implementation verification
#@markdown The importance scores for most probable word (` basketball`) in the sentence `My brother plays football, but my sister prefers<mask>.` using  `roberta-base` model should seem like this:
#@markdown Uncomment the last two lines and compare your result to ours.


# @title visualize importance

colormap = "RdYlGn" #@param ["RdYlGn", "bwr_r"]
alpha = 0.8 #@param {type:"number"}
right_to_left = False #@param {type:"boolean"}

our_tokens = ['My', 'brother', 'plays', 'football', ',', 'but', 'my', 'sister', 'prefers', ' basketball', '.']
our_importances = [0.007425040006637573, 0.0023840665817260742, -0.05275896191596985, 0.2751784808933735, 0.032991498708724976, 0.1144493818283081, 0.017473608255386353, 0.06914246082305908, 0.31883253064006567, 0.0, 0.1255783885717392]

display(visualize_attention([our_tokens], [our_importances], color_maps=colormap, rtl=right_to_left, alpha=alpha))

# your_tokens, your_importances = compute_importances('My brother plays football, but my sister prefers<mask>.', 0)
# display(visualize_attention([your_tokens], [your_importances], color_maps=colormap, rtl=right_to_left, alpha=alpha))

Now that you implemented erasure, it is time to interpret our models.

<b><font color="seaGreen">Interpret a MLM (of your choice) predictions for two of these three `(text, target word)` provided. Use both positive and negative importance scores in your explanations. The first two texts are from LMBADA dataset, and the third is derived from WinoGrad Schema Challenge. If there is another example which you can explain better (containing more valuable information), replace it with one of the those two. </font></b>

### Example #1

**text**: `the battery on logan 's radio must have been on the way out . so he told himself . there was no other explanation beyond cygan and the staff at the white house having been overrun . lizzie opened her eyes with a flutter . they had been on the icy road for an hour without incident . jack was happy to do all of the[MASK].`

**target word**: `driving`

### Example #2

**text**: `he heard rhinna speak `` the queen wants you in her carriage . '' tom spoke `` no , i 'm not going in some asylum . '' ran was seen standing next to him spoke `` it 's just for a private talk with you that 's all . '' tom groaned and went inside the carriage to sit down next to the*.`

**target word**: `queen`

### Example #3
**text**: `The delivery truck zoomed by the school bus because the[MASK] was going so fast.`

**target word**: `truck`

In [ ]:
#@title visualize importances 1

text = "the battery on logan 's radio must have been on the way out . so he told himself . there was no other explanation beyond cygan and the staff at the white house having been overrun . lizzie opened her eyes with a flutter . they had been on the icy road for an hour without incident . jack was happy to do all of the[MASK]." #@param {type:"string"}
target_token_index = 0 #@param {type:"integer"}

colormap = "RdYlGn" #@param ["RdYlGn", "bwr_r"]
alpha = 0.8 #@param {type:"number"}
right_to_left = False #@param {type:"boolean"}

tokens, importances = compute_importances(text, target_token_index)
display(visualize_attention([tokens], [importances], color_maps=colormap, rtl=right_to_left, alpha=alpha))


<font color="red"> <--Write Your Answer Here--> </font>


In [ ]:
#@title visualize importances 2

text = "The delivery truck zoomed by the school bus because the[MASK] was going so fast." #@param {type:"string"}
target_token_index = 0 #@param {type:"integer"}

colormap = "RdYlGn" #@param ["RdYlGn", "bwr_r"]
alpha = 0.8 #@param {type:"number"}
right_to_left = False #@param {type:"boolean"}

tokens, importances = compute_importances(text, target_token_index)
display(visualize_attention([tokens], [importances],
                            color_maps=colormap, rtl=right_to_left, alpha=alpha))


<font color="red"> <--Write Your Answer Here--> </font>


<b><font color="seaGreen">Give an example that the importance scores seem counter-intuitive. (You cannot explain that or you could better explain if the scores were reversed!)
1.  What do you think about it? Does the model realy work in a way that is not *plausible* to humans? Or the interpretation method is not *faithful*? 
2. If you doubt the faithfulness of erasure method (our simplified variant), explain its weaknesses in more details.

</font></b>

In [ ]:
#@title visualize importances (counter-intuitive)

text = "" #@param {type:"string"}
target_token_index = 0 #@param {type:"integer"}

colormap = "RdYlGn" #@param ["RdYlGn", "bwr_r"]
alpha = 0.8 #@param {type:"number"}
right_to_left = False #@param {type:"boolean"}

tokens, importances = compute_importances(text, target_token_index)
display(visualize_attention([tokens], [importances], color_maps=colormap, rtl=right_to_left, alpha=alpha))


<font color="red"> <--Write Your Answer Here--> </font>


# How do Decisions Emerge across Layers?

We have implemented a simple interpretation method in the previous part. Erasure is a feature attribution method, i.e. it attributes the prediction to the input features.
In the last part of this assignment, we are going to learn about one of the most sophisticated (and most interesting) interpretation methods working on transformers. The method is presented in EMNLP 2020 with the [paper](https://www.aclweb.org/anthology/2020.emnlp-main.262) titled: 
> **How do Decisions Emerge across Layers in Neural Models?
Interpretation with Differentiable Masking**

We call the method ***DiffMask***, like the authors did. Thankfully, they release the full source code, so we don't need to implement anything. We just want you to read the paper and deeply understand the concepts, run some code cell and answer some questions. So...

<b><font color="seaGreen">Read the paper and answer the following questions. Just go deep enough to answer the questions and explain the visualized results, not deeper! AND IF YOU HAVE INSOLUBLE AMBIGUITY, BE SURE TO CONTACT TAs.

1. Why the erasure method is susceptible to hindsight bias?
2. What are the advantages of DiffMask over erasure?
3. What was the motivation of designing a toy task?
4. Figures 5, 6 shows a conflict between the two variants of DiffMask on the importance of special tokens! Can you explain what realy happens for `[CLS]` and `[SEP]` in sentiment anaysis?

</font></b>


<font color="red"> <--Write Your Answer Here--> </font>


## Prepare environment to run DiffMask for SQuAD

Let's run some code and see DiffMask in action.

In [ ]:
#@title install libs (restart after installing)
from IPython.display import clear_output

!pip install pytorch-lightning==0.7.3 transformers==2.9.0
!pip install sty
!apt install megatools

!git clone https://github.com/nicola-decao/diffmask
%cd diffmask
!python setup.py install
%cd /content

clear_output()
print('Done! Please reastart runtime.')

In [ ]:
#@title download DiffMask model (takse some time)
# input squad model
!megadl 'https://mega.nz/#!egZTQaLb!Rq6j1KCithdYgxP0aEJAXafPc3_eVS3u8AOjcXQNRF4'

# hidden squad model
# !megadl 'https://mega.nz/#!GtRWhRwC!FV8ZbqVkiO-MtJQPTBKUTye2g0uwZ75wQdVo2lIBJ04'

# sst
# !megadl https://mega.nz/#!bxhgnJDQ!ZiZHzoEDPk4Ub5Cj-f9Yey0ekVynqa7AopwZKwQI-X0
clear_output()
print('Done!')

In [ ]:
#@title import libs

import os
import argparse
import torch
import json
from collections import defaultdict

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

from diffmask.models.question_answering_squad_diffmask import (
    BertQuestionAnsweringSquadDiffMask,
)
from diffmask.models.sentiment_classification_sst_diffmask import (
    BertSentimentClassificationSSTDiffMask,
    RecurrentSentimentClassificationSSTDiffMask,
    PerSampleDiffMaskRecurrentSentimentClassificationSSTDiffMask,
    PerSampleREINFORCERecurrentSentimentClassificationSSTDiffMask,
)
from diffmask.utils.plot import plot_sst_attributions
from diffmask.utils.plot import plot_squad_attributions, print_attributions

INFO:transformers.file_utils:PyTorch version 1.8.1+cu101 available.
INFO:transformers.file_utils:TensorFlow version 2.5.0 available.


In [ ]:
#@title load DiffMask model
device = "cuda"
model = BertQuestionAnsweringSquadDiffMask.load_from_checkpoint(
    '/content/squad-diffmask-input.ckpt').to(device)
model.freeze()

clear_output()
print('Done!')

Now we are ready to go. We have hand-picked a few [SQuAD 1.1](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/) examples for you to ask some question about them. Of course you can experiment with any arbitrary context, question including those from the SQuAD task. If you fully understand the **Figure 8** in the paper (Visualized DiffMask results for SQuAD), continue and answer the following questions:

<b><font color="seaGreen">Try to explain how the model finds the corrects answer in the following example, using DiffMask results. What is ignored and what is considered important? why?
Can you use your insight of the model to make it select "disruptive students" by prepending a piece of text to the context? (Try to manually attack the model)</font></b>

In [ ]:
question = "Who may teachers focus on, in order to prioritize attention?"

attack = ""

context = attack + """Where school class sizes are typically 40 to 50 students, maintaining order \
in the classroom can divert the teacher from instruction, leaving little opportunity for \
concentration and focus on what is being taught. In response, teachers may concentrate \
their attention on motivated students, ignoring attention-seeking and disruptive students. \
The result of this is that motivated students, facing demanding university entrance \
examinations, receive disproportionate resources. Given the emphasis on attainment of \
university places, administrators and governors may regard this policy as appropriate."""

In [ ]:
#@title visual output 1

inputs_dict = {
    k: v.to(device)
    for k, v in model.tokenizer.encode_plus(
        question,
        context,
        max_length=384,
        pad_to_max_length=True,
        return_tensors="pt",
    ).items()
}
inputs_dict["mask"] = inputs_dict["attention_mask"]
del inputs_dict["attention_mask"]

question = model.tokenizer.tokenize(question)
context = model.tokenizer.tokenize(context)
tokens = ["[CLS]"]  + question + ["[SEP]"]  + context + ["[SEP]"]

logits_start_orig, logits_end_orig, expected_L0_full = model.forward_explainer(
    **inputs_dict, attribution=True
)
attributions = expected_L0_full.exp()[0,:len(tokens)].cpu()

plot_squad_attributions(attributions, tokens, context, question,
                        inputs_dict, logits_start_orig, logits_end_orig)


<font color="red"> <--Write Your Answer Here--> </font>


The model gives the wrong answer to the question bellow. (Ground Truth Answers: *the Keraites*, *Keraites*.
Prediction: *Jadaran*)

<b><font color="seaGreen">According to the visualized results of DiffMask:
1. Which parts of the passage causes the model to make mistake?
2. Is the model too far from finding the right answer? Can you help it find the answer with minimum change in the passage?

</font></b>

In [ ]:
question = "What tribe did Toghrul lead?"

context = """Temüjin began his ascent to power by offering himself as an ally (or, \
according to other sources, a vassal) to his father's anda (sworn brother or blood \
brother) Toghrul, who was Khan of the Keraites, and is better known by the Chinese \
title "Wang Khan", which the Jurchen Jin dynasty granted him in 1197. This relationship \
was first reinforced when Börte was captured by the Merkits. Temüjin turned to Toghrul \
for support, and in response, Toghrul offered his vassal 20,000 of his Keraite warriors \
and suggested that he also involve his childhood friend Jamukha, who had himself become \
Khan (ruler) of his own tribe, the Jadaran."""

In [ ]:
#@title visual output 2

inputs_dict = {
    k: v.to(device)
    for k, v in model.tokenizer.encode_plus(
        question,
        context,
        max_length=384,
        pad_to_max_length=True,
        return_tensors="pt",
    ).items()
}
inputs_dict["mask"] = inputs_dict["attention_mask"]
del inputs_dict["attention_mask"]

question = model.tokenizer.tokenize(question)
context = model.tokenizer.tokenize(context)
tokens = ["[CLS]"]  + question + ["[SEP]"]  + context + ["[SEP]"]

logits_start_orig, logits_end_orig, expected_L0_full = model.forward_explainer(
    **inputs_dict, attribution=True
)
attributions = expected_L0_full.exp()[0,:len(tokens)].cpu()

plot_squad_attributions(attributions, tokens, context, question,
                        inputs_dict, logits_start_orig, logits_end_orig)


<font color="red"> <--Write Your Answer Here--> </font>


<b><font color="seaGreen">How can you explain the result of DiffMask for the example below? (the answer is correct) What does it mean when some tokens are considered removable in the middle layers, but are selected as the final answer?</font></b>

In [ ]:
question = "What is the mlolongo system?"

context = """The election held in 1988 saw the advent of the mlolongo (queuing) system, \
where voters were supposed to line up behind their favoured candidates instead of a secret \
ballot. This was seen as the climax of a very undemocratic regime and it led to widespread \
agitation for constitutional reform. Several contentious clauses, including one that \
allowed for only one political party were changed in the following years. In democratic, \
multiparty elections in 1992 and 1997, Daniel arap Moi won re-election."""

In [ ]:
#@title visual output 3

inputs_dict = {
    k: v.to(device)
    for k, v in model.tokenizer.encode_plus(
        question,
        context,
        max_length=384,
        pad_to_max_length=True,
        return_tensors="pt",
    ).items()
}
inputs_dict["mask"] = inputs_dict["attention_mask"]
del inputs_dict["attention_mask"]

question = model.tokenizer.tokenize(question)
context = model.tokenizer.tokenize(context)
tokens = ["[CLS]"]  + question + ["[SEP]"]  + context + ["[SEP]"]

logits_start_orig, logits_end_orig, expected_L0_full = model.forward_explainer(
    **inputs_dict, attribution=True
)
attributions = expected_L0_full.exp()[0,:len(tokens)].cpu()

plot_squad_attributions(attributions, tokens, context, question,
                        inputs_dict, logits_start_orig, logits_end_orig)


<font color="red"> <--Write Your Answer Here--> </font>


# Submission

Congratulations! You finished the assignment & you're ready to submit your work. Please follow the instructions:

1. Check and review your answers. Make sure all of the cell outputs are what you want. 
2. Select File > Save.
3. **Fill your information** & run the cell bellow.
4. Run **Make Submission** cell, It may take several minutes and it may ask you for your credential.
5. Run **Download Submission** cell to obtain your submission as a zip file.
6. Grab the downloaded file (`nlp_asg04__xx__xx.zip`) and hand it over in microsoft teams.

## Fill your information (Run the cell)

In [ ]:
#@title Enter your information & "RUN the cell!!" { run: "auto" }
student_id = "" #@param {type:"string"}
student_name = "" #@param {type:"string"}

print("your student id:", student_id)
print("your name:", student_name)


from pathlib import Path

ASSIGNMENT_PATH = Path('asg04')
ASSIGNMENT_PATH.mkdir(parents=True, exist_ok=True)

## Make Submission (Run the cell)

In [ ]:
#@title Make submission
! pip install -U --quiet PyDrive > /dev/null
! pip install -U --quiet jdatetime > /dev/null

# ! wget -q https://github.com/github/hub/releases/download/v2.10.0/hub-linux-amd64-2.10.0.tgz 


import os
import time
import yaml
import json
import jdatetime

from google.colab import files
from IPython.display import Javascript
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

asg_name = 'NLP_Assignment_4'
script_save = '''
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});
'''
# repo_name = 'iust-deep-learning-assignments'
submission_file_name = 'nlp_asg04__%s__%s.zip'%(student_id, student_name.lower().replace(' ',  '_'))

sub_info = {
    'student_id': student_id,
    'student_name': student_name, 
    'dateime': str(jdatetime.date.today()),
    'asg_name': asg_name
}
json.dump(sub_info, open('info.json', 'w'))

Javascript(script_save)

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_id = drive.ListFile({'q':"title='%s.ipynb'"%asg_name}).GetList()[0]['id']
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('%s.ipynb'%asg_name) 

! jupyter nbconvert --to script "$asg_name".ipynb > /dev/null
! jupyter nbconvert --to html "$asg_name".ipynb > /dev/null
! zip "$submission_file_name" "$asg_name".ipynb "$asg_name".html "$asg_name".txt info.json > /dev/null

print("##########################################")
print("Done! Submisson created, Please download using the bellow cell!")

In [ ]:
drive.ListFile({'q':"title='%s.ipynb'"%asg_name}).GetList()[0]['id']

In [ ]:
files.download(submission_file_name)